In [1]:
import whisper
from pytube import YouTube
import gradio as gr
import os

In [2]:
tiny_model = whisper.load_model('tiny')
base_model = whisper.load_model('base')
small_model = whisper.load_model('small')
medium_model = whisper.load_model('medium')

100%|█████████████████████████████████████| 72.1M/72.1M [00:31<00:00, 2.40MiB/s]
100%|███████████████████████████████████████| 139M/139M [01:31<00:00, 1.58MiB/s]
100%|█████████████████████████████████████| 1.42G/1.42G [11:47<00:00, 2.16MiB/s]


In [3]:
def yt_audio(video_url):
	yt = YouTube(str(video_url))
	video = yt.streams.filter(only_audio=True).first()
	destination = 'Files/'
	audio_file = video.download(output_path=destination)
	base, ext = os.path.splitext(audio_file)
	file_path = base + '.mp3'
	os.rename(audio_file, file_path)
	
	return str(file_path)

In [6]:
def trascribe(model_type ,video_url):

    if model_type == 'Tiny':
        model = tiny_model
    elif model_type == 'Base':
        model = base_model
    elif model_type == 'Small':
        model = small_model
    elif model_type == 'Medium':
        model = medium_model
    
    
    file_path = yt_audio(video_url)

    trascription_dict = model.transcribe(file_path)
    text = trascription_dict['text']
    language = trascription_dict['language']

    return text, language

In [ ]:
examples = [['Princess Elizabeth\'s Speech 1947' (en)'['https://www.youtube.com/watch?v=6yDDqO4dH3c&t=23s']],
            [['BBC News (en)'['https://www.youtube.com/watch?v=9V3L8R5DlNg']]

In [21]:
speech_samples = [  ["Princess Elizabeth\'s Speech 1947' (en)", ['https://www.youtube.com/watch?v=6yDDqO4dH3c&t=23s']],
                    ["Princess Elizgvhbabeth\'s Speech 1947' (en)", ['https://www.youtube.com/watch?v=6yDDqO4dH3c&t=23s']] ]

In [27]:
print(speech_samples[1][1])

['https://www.youtube.com/watch?v=6yDDqO4dH3c&t=23s']


In [29]:
demo_app = gr.Interface(
    title = "Youtube video trascription",
    description = "Youtube video trascription using OpenAI's Whisper models \n\n 1. Select Model, \n 2. Pick Youtube video, copy video URL, \n 3. Paste URL in the Youtube Video URL input box and click 'Submit' button",
    fn = trascribe,
    inputs = [  gr.Dropdown(choices=['Tiny', 'Base' ,'Small', 'Medium'], label="Model", default='Base'),
                gr.Textbox(lines=1, placeholder="Youtube Video URL", label="Video URL"),],

    outputs = [ gr.Textbox(placeholder="Transcritpion.", interactive=True, label="Transcription"),
                gr.Text(placeholder="Detected Language.", interactive=False, label="Language"),],

    #examples = speech_samples,
    
    allow_flagging = "never",
)

demo_app.launch(share=False, inbrowser=True)

/Users/olang/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/gradio/deprecation.py:43: UserWarning: You have unused kwarg parameters in Dropdown, please remove them: {'default': 'Base'}
  warnings.warn(


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f933828dc70>, 'http://127.0.0.1:7863/', None)